In [3]:
from torch import nn
import torch
from torch.utils.tensorboard import SummaryWriter

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.3, shuffle=True, random_state=0)
input_dim = X_train.shape[1]

X_train = torch.Tensor(X_train).to(device)
X_test = torch.Tensor(X_test).to(device)
y_train = torch.Tensor(y_train.reshape(-1, 1)).to(device).float()

In [ ]:
class CancerClassifier(nn.Module):

    def __init__(self, input_dim):
        super(CancerClassifier, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.output = nn.Sigmoid()
    
    def forward(self, x):
        x = self.linear(x)
        x = self.output(x)
        return x

model = CancerClassifier(input_dim)
model.to(device)
print(model)

CancerClassifier(
  (linear): Linear(in_features=30, out_features=1, bias=True)
  (output): Sigmoid()
)


In [ ]:
epochs = 10000
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()
#学習開始
for epoch in range(epochs):
    #勾配を初期化
    optimizer.zero_grad()

    #順伝播
    outputs = model(X_train)

    #誤差を計算
    loss = loss_fn(outputs, y_train)

    #誤差を逆伝播
    loss.backward()

    #重みを修正
    optimizer.step()

    y_pred = model(X_train.float())
    if epoch % 1000 == 0:
        print('Epoch',epoch,'Loss:',loss.item(), '- Pred:', y_pred.data[0])

print("done")

Epoch 0 Loss: 35.68506622314453 - Pred: tensor([5.5249e-23], device='cuda:0')
Epoch 1000 Loss: 0.16222035884857178 - Pred: tensor([0.9681], device='cuda:0')
Epoch 2000 Loss: 0.1300269365310669 - Pred: tensor([0.9852], device='cuda:0')
Epoch 3000 Loss: 0.1110725849866867 - Pred: tensor([0.9915], device='cuda:0')
Epoch 4000 Loss: 0.09868291020393372 - Pred: tensor([0.9946], device='cuda:0')
Epoch 5000 Loss: 0.09013145416975021 - Pred: tensor([0.9963], device='cuda:0')
Epoch 6000 Loss: 0.08391990512609482 - Pred: tensor([0.9973], device='cuda:0')
Epoch 7000 Loss: 0.07930783182382584 - Pred: tensor([0.9979], device='cuda:0')
Epoch 8000 Loss: 0.0758814662694931 - Pred: tensor([0.9983], device='cuda:0')
Epoch 9000 Loss: 0.07324346154928207 - Pred: tensor([0.9987], device='cuda:0')
done


In [6]:
!tensorboard --logdir=runs
writer = SummaryWriter()


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 2036, in shutdown
    h.flush()
  File "/usr/local/lib/python3.7/dist-packages/absl/logging/__init__.py", line 851, in flush
    def flush(self):
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
y_pred = model(X_test)
y_pred = torch.round(y_pred)
y_pred = y_pred.to("cpu").detach().numpy()
print(accuracy_score(y_pred, y_test))

0.9649122807017544
